## 初始化

In [1]:
!git clone https://github.com/HKUDS/LightRAG.git

Cloning into 'LightRAG'...



In [2]:
%cd LightRAG
!pip install -e .

/teamspace/studios/this_studio/LightRAG/basic/LightRAG


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///teamspace/studios/this_studio/LightRAG/basic/LightRAG
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 161.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 227.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 201.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 196.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 310.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!curl https://raw.githubusercontent.com/gusye1234/nano-graphrag/main/tests/mock_data.txt > ./book.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0  1679k      0 --:--:-- --:--:-- --:--:-- 1679k


## model

### openai

In [ ]:
# import os
# import openai

# # 设置API密钥
# openai.api_key = ''

# # 修改 OPENAI_API_BASE 环境变量
# os.environ['OPENAI_API_BASE'] = 'https://api.siliconflow.cn/v1'

# # 验证更改是否生效
# print(f"OpenAI API Base: {os.environ['OPENAI_API_BASE']}")

### ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!ollama pull nomic-embed-text

In [ ]:
!ollama pull qwen2.5-coder:14b

In [9]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete
from lightrag.llm import ollama_model_complete, ollama_embedding
from lightrag.utils import EmbeddingFunc

# Uncomment the below two lines if running in a Jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

WORKING_DIR = "./dickens"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# Initialize LightRAG with Ollama model
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=ollama_model_complete,  # Use Ollama model for text generation
    llm_model_name='qwen2.5-coder:14b',     # Your model name
    # Use Ollama embedding function
    embedding_func=EmbeddingFunc(
        embedding_dim=768,
        max_token_size=8192,
        func=lambda texts: ollama_embedding(
            texts,
            embed_model="nomic-embed-text"
        )
    ),
)

# Define an async function to handle the insertion
async def main():
    with open("./book.txt") as f:
        await rag.ainsert(f.read())

# Run the async function
await main()

# Perform search queries
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="naive")))
# print(rag.query("What are the top themes in this story?", param=QueryParam(mode="local")))
# print(rag.query("What are the top themes in this story?", param=QueryParam(mode="global")))
# print(rag.query("What are the top themes in this story?", param=QueryParam(mode="hybrid")))


INFO:lightrag:Logger initialized for working directory: ./dickens
INFO:lightrag:Load KV llm_response_cache with 0 data
INFO:lightrag:Load KV full_docs with 0 data
INFO:lightrag:Load KV text_chunks with 0 data
INFO:lightrag:Loaded graph from ./dickens/graph_chunk_entity_relation.graphml with 0 nodes, 0 edges
INFO:nano-vectordb:Load (0, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 0 data
INFO:nano-vectordb:Load (0, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Load (42, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 42 data
INFO:lightrag:[New Docs] inserting 1 docs
INFO:lightrag:[New Chunks] inserting 42 chunks
INFO:lightrag:Inserting 42 vectors to chunks
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

The top themes in this story are adventure, love, and friendship.


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 13 relations, 3 text units
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 13 entites, 13 relations, 3 text units


The top themes in this story include love, friendship, and perseverance.


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


The top themes in this story include:

1. **Conflict**: The protagonist faces internal and external struggles throughout the narrative, which drive the plot forward.
2. **Redemption**: Despite initial setbacks, the character undergoes a transformation and achieves redemption by the end of the story.
3. **Identity**: The protagonist's sense of self is tested and redefined as they navigate their journey.

Here is an example code snippet that represents these themes:

```python
class StoryThemes:
    def __init__(self):
        self.themes = [
            "Conflict",
            "Redemption",
            "Identity"
        ]

    def get_top_themes(self):
        return self.themes

story_themes = StoryThemes()
top_themes = story_themes.get_top_themes()

for theme in top_themes:
    print(f"Top Theme: {theme}")
```

This code defines a `StoryThemes` class with an initializer that sets up the themes. The `get_top_themes()` method returns the list of themes. The script creates an instance o

In [21]:
print(rag.query("中文回答，Ghost of Jacob Marley是谁，他的人物关系是什么，至少说出1000字", param=QueryParam(mode="hybrid")))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 5 relations, 3 text units
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 13 entites, 13 relations, 3 text units
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


《圣诞颂歌》是查尔斯·狄更斯的代表作之一，而"Jacob Marley"这个角色是小说中一个非常重要的存在。在故事中，Marley是一个已故的商人，他是主人公Ebenezer Scrooge的旧友和商业对手。

故事开始于圣诞节前夜，Scrooge在家中孤独地度过，他对于节日感到厌烦，并且对人类社会和生活的态度非常消极。就在这个时候，Marley的灵魂突然出现，告诉Scrooge他已经被打入地狱，因为他生前贪心、自私，忽视了家庭和慈善事业。

Marley的灵魂警告Scrooge，在他死后，如果他不改变自己的生活方式，他的灵魂也会被永远困在阴曹地府。同时，Marley还提到，只有通过三名圣诞精灵的指引，Ebenezer才能有机会改正自己的错误，重新获得生活的意义和价值。

Marley的灵魂离开后，Scrooge遇到了第一号圣诞精灵，这个精灵代表着过去，他向Scrooge展示了Scrooge过去的美好时光，包括他与妻子以及老朋友的温馨时光。这些回忆让Scrooge感到了深深的愧疚和后悔。

随后，第二号圣诞精灵出现了，她代表的是现在，她告诉Scrooge关于他目前的生活状况，以及他对周围人的影响。这个精灵让Scrooge看到了自己的自私和冷漠，从而让他意识到自己需要改变。

最后，第三号圣诞精灵登场，她代表着未来，她向Scrooge展示了如果他不改变自己，将会带给周围人以及他自己什么样的后果。这些画面让Scrooge感到了深深的恐惧和担忧。

在经历了这三个精灵的指引后，Ebenezer Scrooge最终决定改正自己的错误，成为一个更善良、更有同情心的人。他开始关心周围的人，尤其是他的员工和家人，并且积极参与慈善事业。

总的来说，Marley的角色在《圣诞颂歌》中起到了非常重要的作用，他是Scrooge改变命运的关键人物。通过与Scrooge的交流，Marley的灵魂让主人公意识到了自己的错误和需要改正的地方。同时，Marley也成为了小说中一个经典的象征，代表了人们对过去、现在和未来的反思以及对生活的重视。

这个故事不仅表达了狄更斯对于社会不公、个人道德和社会责任的批判，同时也传递了一个重要的信息：每个人都应该珍惜自己的生命，关注他人的福祉，并且通过善良的行为来改变自己和社会。而Marley作为Scrooge的朋友和导师，成为了这一主题的重要体现。


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  184k  100  184k    0     0  5435k      0 --:--:-- --:--:-- --:--:-- 5435k
